In [92]:
using ReverseDiff
using ReverseDiff: TrackedArray, gradient, hessian, hessian!, gradient!
using StaticArrays
import Random
using BenchmarkTools: @btime
rng = Random.MersenneTwister()

MersenneTwister(0x94174fffd26f936b2a8d815627ee3249)

In [119]:
import LinearAlgebra as la

In [104]:
positions = (Random.rand(rng, Float64, (3, 10)) .- 0.5) .* 2

3×10 Matrix{Float64}:
  0.865506  -0.107364   0.397068   …   0.475067   0.158897   0.654857
  0.80562   -0.354114   0.0442495     -0.279549  -0.292776   0.729377
 -0.286013   0.486449  -0.0635372      0.545754   0.251755  -0.261811

In [105]:
function SimpleG(positions)
    res = 1
    for pos in eachcol(positions)
        res *= exp(-0.5 * sum(pos.^2))
    end
    return res
end

SimpleG (generic function with 1 method)

In [106]:
config = ReverseDiff.HessianConfig(positions)
tape = ReverseDiff.HessianTape(SimpleG, positions, config)
tape = ReverseDiff.compile(tape)

typename(ReverseDiff.CompiledTape)(SimpleG)

In [107]:
tmp = SizedMatrix{30, 30}(zeros(30, 30));

In [121]:
function kineticRD(result, tape, positions)
    result = ReverseDiff.hessian!(result, tape, positions);
    return -0.5 * sum(la.diag(result)) / SimpleG(positions)
end;

In [122]:
@time hessian!(tmp, tape, positions);

  0.000376 seconds


In [125]:
@time kineticRD(tmp, tape, positions)

  0.000376 seconds (12 allocations: 1.438 KiB)


10.8112957394167

In [ ]:
379.500 μs (0 allocations: 0 bytes)